In [10]:
import csv
import pandas as pd
import os
from tqdm import tqdm
import subprocess
path = 'C:/Users/6997619/Documents/GitHub projects/DHZW-simulation_Sim-2APL/src/main/resources/calibration/'

## Read datasets

In [11]:
X_train = pd.read_csv(path+"data/X_train.csv")
X_test = pd.read_csv(path+"data/X_test.csv")

with open(path+'data/parameter_labels.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    params = [row['parameter'] for row in reader]

## Create datasets with all the parameters

In [12]:
X_train_full = pd.DataFrame()
for label in params:
    X_train_full[label] = [1.0] * X_train.shape[0]

X_test_full = pd.DataFrame()
for label in params:
    X_test_full[label] = [1.0] * X_test.shape[0]

In [13]:
X_train_full.update(X_train)
X_test_full.update(X_test)

In [14]:
table = pd.concat([X_train_full, X_test_full])

In [15]:
table.to_csv(path + "data/parametersets.csv", index=False)

### Initialise the output dataframe

In [16]:
df = pd.DataFrame(columns=["CAR_DRIVER", "CAR_PASSENGER", "BIKE", "BUS_TRAM", "TRAIN", "WALK"])
df.to_csv(path + "output/output_proportions.csv", index=False)

## Call Sim2APL iteratively

### Prepare the Java command

In [17]:
config = "--config src/main/resources/config_DHZW_full.toml"
modeliberal = "--mode-liberal 0.5"
modeconservative = "--mode-conservative 0.5"
output = "-o output/deskrun"
outputPath = " --output_file src/main/resources/calibration/output/output_proportions.csv"
parametersPath = "--parameter_file src/main/resources/calibration/data/parametersets.csv"

cmd = "java -cp target/sim2apl-dhzw-simulation-1.0-SNAPSHOT-jar-with-dependencies.jar main.java.nl.uu.iss.ga.Simulation" + " " + config + " " + modeliberal + " " + modeconservative + " " + output + " " + outputPath + " " + parametersPath
os.chdir('C:/Users/6997619/Documents/GitHub projects/DHZW-simulation_Sim-2APL')

### Main loop call simulations

In [18]:
for idx, row in tqdm(table.iterrows(), total=table.shape[0]):
    # Construct the command line argument for this parameter value
    arg = f'--parameterset_index={idx+1}'
            
    # Construct the full command with the argument
    full_command = f'{cmd} {arg}'
               
    # Capture the output of the Java program
    try:
        output = subprocess.check_output(full_command, stderr=subprocess.STDOUT, universal_newlines=True)
    except subprocess.CalledProcessError as e:
        print(f"Java program exited with non-zero return code: {e.returncode}")
        print(f"Error message: {e.output}")
        exit(1)

 25%|███████████████████▍                                                           | 54/220 [28:50<1:28:38, 32.04s/it]


KeyboardInterrupt: 